# Install Taipy

To install Taipy, just `pip install` it.

In [ ]:
%pip install taipy -q

In [ ]:
%pip install pmdarima -q

In [1]:
pip list

Package                Version
---------------------- -----------
alembic                1.8.1
aniso8601              9.0.1
apispec                5.2.2
apispec-webframeworks  0.5.2
appdirs                1.4.4
argon2-cffi            21.3.0
argon2-cffi-bindings   21.2.0
asttokens              2.0.8
attrs                  22.1.0
backcall               0.2.0
beautifulsoup4         4.11.1
bidict                 0.22.0
bleach                 5.0.1
certifi                2022.9.14
cffi                   1.15.1
charset-normalizer     2.1.1
click                  8.1.3
cloudpickle            2.2.0
cmdstanpy              1.0.8
colorama               0.4.5
contourpy              1.0.5
convertdate            2.4.0
cycler                 0.11.0
Cython                 0.29.32
dask                   2022.12.1
debugpy                1.6.3
decorator              5.1.1
deepdiff               6.2.3
defusedxml             0.7.1
distlib                0.3.4
distributed            2022.12.1
dnspython     

# Import the packages

In [2]:
from taipy.gui import Gui, Markdown, notify
from taipy import Config, Scope
import taipy as tp

import datetime as dt

from pmdarima import auto_arima

from sklearn.linear_model import LinearRegression

import pandas as pd
import numpy as np

# Taipy Gui Basics
## Markdown Syntax

Taipy uses the Markdown syntax to display elements. `#` creates a title, `*` puts your text in italics and `**` puts it in bold.

![](img/gui_basic_eng.png)

In [3]:
page_md = """
# Taipy

Test **here** to put some *markdown*

Click to access the [doc](https://docs.taipy.io/en/latest/)
"""

In [4]:
Gui(page_md).run(dark_mode=False, run_browser=False, port=6001)

[2023-02-17 13:05:23,683][Taipy][INFO] Running in 'single_client' mode in notebook environment
[2023-02-17 13:05:23,938][Taipy][INFO]  * Server starting on http://127.0.0.1:6001


## Visual elements
Create different visual elements. The syntax is always the same for each visual element.  `<|{value}|name_of_visual_element|property_1=value_of_property_1|...|>`
- Create a [slider](https://docs.taipy.io/en/latest/manuals/gui/viselements/slider/) `<|{value}|slider|>`

- Create a [date](https://docs.taipy.io/en/latest/manuals/gui/viselements/date/) `<|{value}|date|>`

- Create a [selector](https://docs.taipy.io/en/latest/manuals/gui/viselements/selector/) `<|{value}|selector|lov={list_of_values}|>`


![](img/control.png)

In [5]:
slider_value = 0
date_value = None
selected_value = None
selector = ['Test 1', 'Test 2', 'Test 3']

control_md = """
## Controls

<|{slider_value}|slider|> <|{slider_value}|>

<|{date_value}|date|> <|{date_value}|>

<|{selected_value}|selector|lov={selector}|> <|{selected_value}|>
"""

In [6]:
Gui(control_md).run(dark_mode=False, run_browser=False)

[2023-02-17 13:05:35,544][Taipy][INFO] Running in 'single_client' mode in notebook environment
[2023-02-17 13:05:35,560][Taipy][INFO]  * Server starting on http://127.0.0.1:5000


## Data Viz

A dataset gathering information on the number of deaths, confirmed cases and recovered for different regions is going to be used to create an interactive Dashboard.

In [8]:
path_to_data = "data/covid-19-all.csv"
data = pd.read_csv(path_to_data, low_memory=False)
data[-5:]

,Country/Region,Province/State,Latitude,Longitude,Confirmed,Recovered,Deaths,Date
1241947,Vietnam,NaN,14.058324,108.277199,1465.0,1325.0,35.0,2020-12-31
1241948,West Bank and Gaza,NaN,31.952200,35.233200,138004.0,117183.0,1400.0,2020-12-31
1241949,Yemen,NaN,15.552727,48.516388,2099.0,1394.0,610.0,2020-12-31
1241950,Zambia,NaN,-13.133897,27.849332,20725.0,18660.0,388.0,2020-12-31
1241951,Zimbabwe,NaN,-19.015438,29.154857,13867.0,11250.0,363.0,2020-12-31


In [9]:
def initialize_case_evolution(data, selected_country='India') -> pd.DataFrame:
    # Aggregation of the dataframe per Country/Region
    country_date_df = data.groupby(["Country/Region",'Date']).sum().reset_index()
    
    # a country is selected, here India by default
    country_date_df = country_date_df.loc[country_date_df['Country/Region']==selected_country]
    return country_date_df

In [10]:
country_date_df = initialize_case_evolution(data)
country_date_df.head()

,Country/Region,Date,Latitude,Longitude,Confirmed,Recovered,Deaths
23021,India,2020-01-30,23.746783,78.96288,1.0,0.0,0.0
23022,India,2020-01-31,23.746783,78.96288,1.0,0.0,0.0
23023,India,2020-02-01,23.746783,78.96288,1.0,0.0,0.0
23024,India,2020-02-02,23.746783,78.96288,2.0,0.0,0.0
23025,India,2020-02-03,23.746783,78.96288,3.0,0.0,0.0


Create a [chart](https://docs.taipy.io/en/latest/manuals/gui/viselements/chart/) showing the evolution of Deaths in France (_Deaths_ for _y_ and _Date_ for _x_). The visual element (chart) has the same syntax as the other ones with specific properties (_x_, _y_, _type_ for example). Here are some [examples of charts](https://docs.taipy.io/en/release-1.1/manuals/gui/viselements/charts/bar/). The _x_ and _y_ porperties only needs the name of the dataframe columns to display.

![](img/simple_graph.png)

In [11]:
country_md = "<|{country_date_df}|chart|x=Date|y=Deaths|type=bar|>"

In [12]:
Gui(country_md).run(dark_mode=False, run_browser=False, port=6005)

[2023-02-17 13:06:27,149][Taipy][INFO] Running in 'single_client' mode in notebook environment
[2023-02-17 13:06:27,166][Taipy][INFO]  * Server starting on http://127.0.0.1:6005


## Add new traces

- Add on the graph the number of Confirmed and Recovered cases (_Confirmed_ and _Recovered_) with the number of Deaths
- _y_ (and _x_) can be indexed this way to add more traces (`y[1]=`, `y[2]=`, `y[3]=`).

![](img/multi_traces.png)

In [13]:
country_md = "<|{country_date_df}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|>"

In [14]:
Gui(country_md).run(dark_mode=False, run_browser=False)

[2023-02-17 13:06:40,818][Taipy][INFO] Running in 'single_client' mode in notebook environment
[2023-02-17 13:06:42,903][Taipy][INFO] Gui server has been stopped.
[2023-02-17 13:06:42,904][Taipy][INFO]  * Server starting on http://127.0.0.1:5000


## Style the graph with personalized properties
The _layout_ dictionnary specifies how bars should be displayed. They would be 'stacked'.

The _options_ dictionary will change the opacity of the unselected markers.

These are Plotly properties.

![](img/stack_chart.png)

In [15]:
layout = {'barmode':'stack'}
country_md = "<|{country_date_df}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|>"

In [16]:
Gui(country_md).run(dark_mode=False, run_browser=False, port=6008)

[2023-02-17 13:07:14,725][Taipy][INFO] Running in 'single_client' mode in notebook environment
[2023-02-17 13:07:14,741][Taipy][INFO]  * Server starting on http://127.0.0.1:6008


## Add texts that sums up the data

Use the [text](https://docs.taipy.io/en/latest/manuals/gui/viselements/text/) visual element.

- Add the total number of Deaths (last line of _country_date_df_)
- Add the total number of Recovered (last line of _country_date_df_)
- Add the total number of Confirmed (last line of _country_date_df_)


In [17]:
country_date_df

,Country/Region,Date,Latitude,Longitude,Confirmed,Recovered,Deaths
23021,India,2020-01-30,23.746783,78.962880,1.0,0.0,0.0
23022,India,2020-01-31,23.746783,78.962880,1.0,0.0,0.0
23023,India,2020-02-01,23.746783,78.962880,1.0,0.0,0.0
23024,India,2020-02-02,23.746783,78.962880,2.0,0.0,0.0
23025,India,2020-02-03,23.746783,78.962880,3.0,0.0,0.0
...,...,...,...,...,...,...,...
23353,India,2020-12-27,854.924665,3023.983447,10207871.0,9782669.0,147901.0
23354,India,2020-12-28,854.924665,3023.983447,10224303.0,9807569.0,148153.0
23355,India,2020-12-29,854.924665,3023.983447,10244852.0,9834141.0,148439.0
23356,India,2020-12-30,854.924665,3023.983447,10266674.0,9860280.0,148738.0


This is how we can get the total number of Deaths from the dataset for India.

In [18]:
country_date_df.iloc[-1, 6] # gives the number of deaths for India (5 is for recovered and 4 is confirmed)

148738.0

Use the [text](https://docs.taipy.io/en/release-1.1/manuals/gui/viselements/text/) visual element. Note that between `{}`, any Python variable can be put but also any Python code.

![](img/control_text.png)

In [19]:
country_md = """
## Deaths <|{country_date_df.iloc[-1, 6]}|text|>

## Recovered <|{country_date_df.iloc[-1, 5]}|text|>

## Confirmed <|{country_date_df.iloc[-1, 4]}|text|>

<|{country_date_df}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|>
"""

In [20]:
Gui(country_md).run(dark_mode=False, run_browser=False)

[2023-02-17 13:07:27,148][Taipy][INFO] Running in 'single_client' mode in notebook environment
[2023-02-17 13:07:29,258][Taipy][INFO] Gui server has been stopped.
[2023-02-17 13:07:29,259][Taipy][INFO]  * Server starting on http://127.0.0.1:5000


--- 3 warning(s) were found for page '/' in variable 'country_md' ---
 - Warning 1: Variable 'options' is not available in either the '__main__' or '__main__' modules.
 - Warning 2: Variable 'options' is not defined (in expression '{options}'): 'types.SimpleNamespace' object has no attribute 'options'
 - Warning 3: Cannot evaluate expression 'options': name 'options' is not defined
----------------------------------------------------------------------



## Local _on_change_

- Add a [selector](https://docs.taipy.io/en/latest/manuals/gui/viselements/selector/) with `dropdown=True` containing the name of all the _Country/region_
- Give to the _on_change_ selector property the name of the _on_change_country_ function. This function will be called when the selector will be used.
- This function has a 'state' parameter and has to be completed. When the selector is used, this function is called with the _state_ argument. It contains all the Gui variables; 'state.country_date_df' is then the dataframe used in the Gui.

![](img/on_change_local.png)

In [21]:
country_lov = sorted(data["Country/Region"].dropna().unique().tolist())
selected_country = "India"

country_md = """
<|{selected_country}|selector|lov={country_lov}|on_change=on_change_country|dropdown|label=Country|>

## Deaths <|{country_date_df.iloc[-1, 6]}|>

## Recovered <|{country_date_df.iloc[-1, 5]}|>

## Confirmed <|{country_date_df.iloc[-1, 4]}|>

<|{country_date_df}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|>
"""

In [22]:
def on_change_country(state):
    # state contains all the Gui variables and this is through this state variable that we can update the Gui
    # state.selected_country, state.country_date_df, ...
    # update country_date_df with the right country (use initialize_case_evolution)
    print("Chosen country: ", state.selected_country)
    state.country_date_df = initialize_case_evolution(data, state.selected_country)

In [23]:
Gui(country_md).run(dark_mode=False, run_browser=False, port=6010)

[2023-02-17 13:08:00,551][Taipy][INFO] Running in 'single_client' mode in notebook environment


INFO:Taipy:Running in 'single_client' mode in notebook environment


[2023-02-17 13:08:00,568][Taipy][INFO]  * Server starting on http://127.0.0.1:6010


INFO:Taipy: * Server starting on http://127.0.0.1:6010


Chosen country:  Iran


Chosen country:  Israel


## Layout

Use the [layout](https://docs.taipy.io/en/latest/manuals/gui/viselements/layout/) block to change the page structure. This block creates invisible columns to put text/visual elements in.

Syntax :
```
<|layout|columns=1 1 1 ...|
(first column)

(in second column)

(third column)
(again, third column)

(...)
|>
```

In [26]:
final_country_md = """
<|layout|columns=1 1 1 1|
<|{selected_country}|selector|lov={country_lov}|on_change=on_change_country|dropdown|label=Country|>

## Deaths <|{country_date_df.iloc[-1, 6]}|>

## Recovered <|{country_date_df.iloc[-1, 5]}|>

## Confirmed <|{country_date_df.iloc[-1, 4]}|>
|>

<|{country_date_df}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|>
"""

In [27]:
Gui(final_country_md).run(dark_mode=False, run_browser=False, port=6013)

[2023-02-17 13:08:48,538][Taipy][INFO] Running in 'single_client' mode in notebook environment


INFO:Taipy:Running in 'single_client' mode in notebook environment


[2023-02-17 13:08:50,658][Taipy][INFO] Gui server has been stopped.


INFO:Taipy:Gui server has been stopped.


[2023-02-17 13:08:50,659][Taipy][INFO]  * Server starting on http://127.0.0.1:6013


INFO:Taipy: * Server starting on http://127.0.0.1:6013
'types.SimpleNamespace' object has no attribute '_TpLv_tpec_TpExPr_selected_country_TPMDL_0'
'types.SimpleNamespace' object has no attribute '_TpLv_tpec_TpExPr_selected_country_TPMDL_0'


Chosen country:  Indonesia


![](img/layout.png)

# Map

In [28]:
def initialize_map(data):
    data['Province/State'] = data['Province/State'].fillna(data["Country/Region"])
    data_province = data.groupby(["Country/Region",
                                  'Province/State',
                                  'Longitude',
                                  'Latitude'])\
                         .max()

    data_province_displayed = data_province[data_province['Deaths']>10].reset_index()

    data_province_displayed['Size'] = np.sqrt(data_province_displayed.loc[:,'Deaths']/data_province_displayed.loc[:,'Deaths'].max())*80 + 3
    data_province_displayed['Text'] = data_province_displayed.loc[:,'Deaths'].astype(str) + ' deaths</br>' + data_province_displayed.loc[:,'Province/State']
    return data_province_displayed

In [29]:
data_province_displayed = initialize_map(data)
data_province_displayed.head()

,Country/Region,Province/State,Longitude,Latitude,Confirmed,Recovered,Deaths,Date,Size,Text
0,Afghanistan,Afghanistan,67.709953,33.93911,51526.0,41727.0,2191.0,2020-12-31,17.771247,2191.0 deaths</br>Afghanistan
1,Albania,Albania,20.168300,41.15330,58316.0,33634.0,1181.0,2020-12-31,13.844784,1181.0 deaths</br>Albania
2,Algeria,Algeria,1.659600,28.03390,99610.0,67127.0,2756.0,2020-12-31,19.566684,2756.0 deaths</br>Algeria
3,Andorra,Andorra,1.521800,42.50630,8049.0,7432.0,84.0,2020-12-31,5.892249,84.0 deaths</br>Andorra
4,Angola,Angola,17.873900,-11.20270,17553.0,11044.0,405.0,2020-12-31,9.350728,405.0 deaths</br>Angola


Properties to style the map
- marker color corresponds to the number of Deaths (column _Deaths_)
- marker sizes corresponds to the size in _Size_ column which is created from the number of Deaths

layout_map permet defined the initial zoom and position of the map


In [30]:
marker_map = {"color":"Deaths", "size": "Size", "showscale":True, "colorscale":"Viridis"}
layout_map = {
            "dragmode": "zoom",
            "mapbox": { "style": "open-street-map", "center": { "lat": 38, "lon": -90 }, "zoom": 3}
            }

We give to Plotly:
- a map type
- the name of the latitude column
- the name of the longitude column
- properties: on the size and color of the markers
- the name of the column for the text of the points

In [31]:
selected_points = []
map_md = """
<|{data_province_displayed}|chart|type=scattermapbox|selected={selected_points}|lat=Latitude|lon=Longitude|marker={marker_map}|layout={layout_map}|text=Text|mode=markers|height=800px|options={options}|>
"""

In [36]:
Gui(map_md).run(dark_mode=False, run_browser=False, port=6017)

[2023-02-17 16:57:28,454][Taipy][INFO] Running in 'single_client' mode in notebook environment


INFO:Taipy:Running in 'single_client' mode in notebook environment


[2023-02-17 16:57:30,778][Taipy][INFO] Gui server has been stopped.


INFO:Taipy:Gui server has been stopped.


[2023-02-17 16:57:30,780][Taipy][INFO]  * Server starting on http://127.0.0.1:6017


INFO:Taipy: * Server starting on http://127.0.0.1:6017
--- 3 warning(s) were found for page '/' in variable 'map_md' ---
 - Warning 1: Variable 'options' is not available in either the '__main__' or '__main__' modules.
 - Warning 2: Variable 'options' is not defined (in expression '{options}'): 'types.SimpleNamespace' object has no attribute 'options'
 - Warning 3: Cannot evaluate expression 'options': name 'options' is not defined
------------------------------------------------------------------



![](img/carte.png)

# Part and the _render_ property
- Create a [toggle](https://docs.taipy.io/en/latest/manuals/gui/viselements/toggle/) (works the same as a selector) with a lov of 'Map' an 'Country'
- Create two part blocks that renders or not depending on the value of the toggle
    - To do this, use the fact that in the _render_ property of the part block, Python code can be inserted in `{}`

In [33]:
representation_toggle = ['Map', 'Country']
selected_representation = representation_toggle[0]

In [34]:
main_page = """
<|{selected_representation}|toggle|lov={representation_toggle}|>

<|part|render={selected_representation == "Country"}|
"""+final_country_md+"""
|>

<|part|render={selected_representation == "Map"}|
"""+map_md+"""
|>
""" 

In [35]:
Gui(main_page).run(dark_mode=False, run_browser=False, port=6019)

[2023-02-17 13:10:35,438][Taipy][INFO] Running in 'single_client' mode in notebook environment


INFO:Taipy:Running in 'single_client' mode in notebook environment


[2023-02-17 13:10:35,455][Taipy][INFO]  * Server starting on http://127.0.0.1:6019


INFO:Taipy: * Server starting on http://127.0.0.1:6019


![](img/part_render.png)

# Taipy Core
Here are the functions that we are going to use to predict the number of Deaths for a country.
We will:
- preprocess the data (_preprocess_),
- create a training and testing database (_make_train_test_data_),
- train a model (_train_model_),
- generate predictions (_forecast_),
- generate a dataframe with the historical data and the predictions (_result_)

![](img/all_architecture.svg)

In [ ]:
# initialise variables
selected_scenario = None
scenario_selector = None

first_date = dt.datetime(2020,11,1)

scenario_name = None

result = None

In [ ]:
#Config.configure_job_executions(mode="standalone", nb_of_workers=2)

In [ ]:

def add_features(data):
    dates = pd.to_datetime(data["Date"])
    data["Months"] = dates.dt.month
    data["Days"] = dates.dt.isocalendar().day
    data["Week"] = dates.dt.isocalendar().week
    data["Day of week"] = dates.dt.dayofweek
    return data

def create_train_data(final_data, date):
    bool_index = pd.to_datetime(final_data['Date']) <= date
    train_data = final_data[bool_index]
    return train_data

def preprocess(initial_data, country, date):
    data = initial_data.groupby(["Country/Region",'Date'])\
                       .sum()\
                       .dropna()\
                       .reset_index()

    final_data = data.loc[data['Country/Region']==country].reset_index(drop=True)
    final_data = final_data[['Date','Deaths']]
    final_data = add_features(final_data)
    
    train_data = create_train_data(final_data, date)
    return final_data, train_data


def train_arima(train_data):
    model = auto_arima(train_data['Deaths'],
                       start_p=1, start_q=1,
                       max_p=5, max_q=5,
                       start_P=0, seasonal=False,
                       d=1, D=1, trace=True,
                       error_action='ignore',  
                       suppress_warnings=True)
    model.fit(train_data['Deaths'])
    return model


def forecast(model):
    predictions = model.predict(n_periods=60)
    return np.array(predictions)


def result(final_data, predictions, date):
    dates = pd.to_datetime([date + dt.timedelta(days=i)
                            for i in range(len(predictions))])
    final_data['Date'] = pd.to_datetime(final_data['Date'])
    predictions = pd.concat([pd.Series(dates, name="Date"),
                             pd.Series(predictions, name="Predictions")], axis=1)
    return final_data.merge(predictions, on="Date", how="outer")


def train_linear_regression(train_data):    
    y = train_data['Deaths']
    X = train_data.drop(['Deaths','Date'], axis=1)
    
    model = LinearRegression()
    model.fit(X,y)
    return model

def forecast_linear_regression(model, date):
    dates = pd.to_datetime([date + dt.timedelta(days=i)
                            for i in range(60)])
    X = add_features(pd.DataFrame({"Date":dates}))
    X.drop('Date', axis=1, inplace=True)
    predictions = model.predict(X)
    return predictions

First we must define the Data Nodes then the tasks (associated to the Python function). Furthermore, we gather these tasks into different pipelines and these pipelines into a scenario.

A Data Node needs a **unique id**. If needed, the storage type can be changed for CSV and SQL. Other parameters are then needed.

### Data Nodes and Task for preprocess

<img src="img/preprocess.svg" alt="drawing" width="500"/>

In [ ]:
initial_data_cfg = Config.configure_data_node(id="initial_data",
                                              storage_type="csv",
                                              path=path_to_data,
                                              cacheable=True,
                                              validity_period=dt.timedelta(days=5),
                                              scope=Scope.GLOBAL)

country_cfg = Config.configure_data_node(id="country", default_data="India",
                                         cacheable=True, validity_period=dt.timedelta(days=5))


date_cfg = Config.configure_data_node(id="date", default_data=dt.datetime(2020,10,10),
                                         cacheable=True, validity_period=dt.timedelta(days=5))

<img src="img/preprocess.svg" alt="drawing" width="500"/>

In [ ]:
final_data_cfg =  Config.configure_data_node(id="final_data",
                                            cacheable=True, validity_period=dt.timedelta(days=5))


train_data_cfg =  Config.configure_data_node(id="train_data", cacheable=True, validity_period=dt.timedelta(days=5))


<img src="img/preprocess.svg" alt="drawing" width="500"/>

In [ ]:
task_preprocess_cfg = Config.configure_task(id="task_preprocess_data",
                                           function=preprocess,
                                           input=[initial_data_cfg, country_cfg, date_cfg],
                                           output=[final_data_cfg,train_data_cfg])

### Data Nodes and Task for train_model

<img src="img/train_model.svg" alt="drawing" width="500"/>

In [ ]:
model_cfg = Config.configure_data_node(id="model", cacheable=True, validity_period=dt.timedelta(days=5), scope=Scope.PIPELINE)

task_train_cfg = Config.configure_task(id="task_train",
                                      function=train_arima,
                                      input=train_data_cfg,
                                      output=model_cfg)

### Data Nodes and Task for forecast

<img src="img/forecast_arima.svg" alt="drawing" width="500"/>

In [ ]:
predictions_cfg = Config.configure_data_node(id="predictions", scope=Scope.PIPELINE)

task_forecast_cfg = Config.configure_task(id="task_forecast",
                                      function=forecast,
                                      input=model_cfg,
                                      output=predictions_cfg)

### Data Nodes and Task for result

<img src="img/result.svg" alt="drawing" width="500"/>

In [ ]:
result_cfg = Config.configure_data_node(id="result", scope=Scope.PIPELINE)

task_result_cfg = Config.configure_task(id="task_result",
                                      function=result,
                                      input=[final_data_cfg, predictions_cfg, date_cfg],
                                      output=result_cfg)

## [Configuration of pipelines](https://docs.taipy.io/en/release-1.1/manuals/reference/taipy.Config/#taipy.core.config.config.Config.configure_default_pipeline)

In [ ]:
pipeline_preprocessing_cfg = Config.configure_pipeline(id="pipeline_preprocessing",
                                                       task_configs=[task_preprocess_cfg])

pipeline_arima_cfg = Config.configure_pipeline(id="ARIMA",
                                               task_configs=[task_train_cfg,
                                                             task_forecast_cfg,
                                                             task_result_cfg])

## Add more models

<img src="img/pipeline_linear_regression.svg" alt="drawing" width="500"/>

In [ ]:
def train_linear_regression(train_data):    
    y = train_data['Deaths']
    X = train_data.drop(['Deaths','Date'], axis=1)
    
    model = LinearRegression()
    model.fit(X,y)
    return model

def forecast_linear_regression(model, date):
    dates = pd.to_datetime([date + dt.timedelta(days=i)
                            for i in range(60)])
    X = add_features(pd.DataFrame({"Date":dates}))
    X.drop('Date', axis=1, inplace=True)
    predictions = model.predict(X)
    return pd.Series(predictions)


task_train_linear_cfg = Config.configure_task(id="task_train_linear",
                                      function=train_linear_regression,
                                      input=train_data_cfg,
                                      output=model_cfg)

task_forecast_linear_cfg = Config.configure_task(id="task_forecast_linear",
                                      function=forecast_linear_regression,
                                      input=[model_cfg, date_cfg],
                                      output=predictions_cfg)

pipeline_linear_regression_cfg = Config.configure_pipeline(id="LinearRegression",
                                               task_configs=[task_train_linear_cfg,
                                                             task_forecast_linear_cfg,
                                                             task_result_cfg])

## [Configuration of scénario](https://docs.taipy.io/en/release-1.1/manuals/reference/taipy.Config/#taipy.core.config.config.Config.configure_default_scenario)

In [ ]:
scenario_cfg = Config.configure_scenario(id='scenario', pipeline_configs=[pipeline_preprocessing_cfg,
                                                                          pipeline_arima_cfg,
                                                                          pipeline_linear_regression_cfg])

## Creation and submit of scenario

In [ ]:
scenario = tp.create_scenario(scenario_cfg, name='First Scenario')
tp.submit(scenario)

In [ ]:
scenario.initial_data.read()

In [ ]:
scenario.train_data.read()

In [ ]:
scenario.ARIMA.predictions.read()

## Caching
Some job are skipped because no change has been done to the "input" Data Nodes.

In [ ]:
tp.submit(scenario)

## Write in data nodes

To write a data node:

`<Data Node>.write(new_value)`

In [ ]:
scenario.country.write('US')
tp.submit(scenario)
scenario.result.read()

In [ ]:
scenario.ARIMA.predictions.read()

## Simple framework

In [ ]:
scenario = tp.create_scenario(scenario_cfg, name='Second Scenario')
tp.submit(scenario)

In [ ]:
scenario.ARIMA.task_forecast.function

In [ ]:
scenario.ARIMA.model.read()

In [ ]:
scenario.pipelines['LinearRegression'].model.read()

In [ ]:
[s.country.read() for s in tp.get_scenarios()]

In [ ]:
[s.date.read() for s in tp.get_scenarios()]

## Create a Gui for the backend
_scenario_selector_ lets you choose a scenario and display its results.

In [ ]:
scenario_selector = [(s.id, s.name) for s in tp.get_scenarios()]
selected_scenario = scenario.id
print(scenario_selector,'\n', selected_scenario)

In [ ]:
result_arima = scenario.pipelines['ARIMA'].result.read()
result_rd = scenario.pipelines['LinearRegression'].result.read()
result = result_rd.merge(result_arima, on="Date", how="outer").sort_values(by='Date')
result

**Tips** : the _value_by_id_ property if set to True for a selected will make _selected_scenario_ directly refer to the first element of the tupple (here the id)

![](img/predictions.png)

In [ ]:
prediction_md = """
<|layout|columns=1 2 5 1 3|
<|{scenario_name}|input|label=Name|>

<br/>
<|Create|button|on_action=create_new_scenario|>

Prediction date
<|{first_date}|date|>

<|{selected_country}|selector|lov={country_lov}|dropdown|on_change=on_change_country|label=Country|>

<br/>
<|Submit|button|on_action=submit_scenario|>

<|{selected_scenario}|selector|lov={scenario_selector}|on_change=actualize_graph|dropdown|value_by_id|label=Scenario|>
|>

<|{result}|chart|x=Date|y[1]=Deaths_x|type[1]=bar|y[2]=Predictions_x|y[3]=Predictions_y|>
"""

In [ ]:
def create_new_scenario(state):
    scenario = tp.create_scenario(scenario_cfg, name=state.scenario_name)
    state.scenario_selector += [(scenario.id, scenario.name)]

In [ ]:
def submit_scenario(state):
    # 1) get the selected scenario
    # 2) write in country Data Node, the selected country
    # 3) submit the scenario
    # 4) actualize le graph avec actualize_graph
    scenario = tp.get(state.selected_scenario)
    scenario.country.write(state.selected_country)
    scenario.date.write(state.first_date.replace(tzinfo=None))
    tp.submit(scenario)
    actualize_graph(state)

In [ ]:
def actualize_graph(state):
    # 1) update the result dataframe
    # 2) change selected_country with the predicted country of the scenario
    scenario = tp.get(state.selected_scenario)
    result_arima = scenario.pipelines['ARIMA'].result.read()
    result_rd = scenario.pipelines['LinearRegression'].result.read()
    if result_arima is not None and result_rd is not None:
        state.result = result_rd.merge(result_arima, on="Date", how="outer").sort_values(by='Date')
    state.selected_country = scenario.country.read()

In [ ]:
Gui(prediction_md).run(dark_mode=False, port=5090)

# Multi-pages and Taipy Rest

To create a multi-pages app, we only need a dictionary with names as the keys and the Markdowns as the values.

The _navbar_ control (<|navbar|>) has a default behaviour. It redirects to the different pages of the app automatically. Other solutions exists.

![](img/multi_pages.png)

In [ ]:
navbar_md = "<center>\n<|navbar|>\n</center>"

pages = {
    "Map":navbar_md+map_md,
    "Country":navbar_md+final_country_md,
    "Predictions":navbar_md+prediction_md
}

rest = tp.Rest()

gui_multi_pages = Gui(pages=pages)
tp.run(gui_multi_pages, rest, dark_mode=False, port=6066)